In [2]:
import os
from IPython.display import display, HTML
from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.sql.types import LongType
from pyspark.sql.types import IntegerType, DoubleType, ArrayType
from pyspark.sql.functions import regexp_replace
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from pyspark.sql.functions import col, split, expr
from pyspark.sql.functions import udf, lit, collect_list, collect_set
import findspark
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from pyspark.sql.functions import col, split, expr
from pyspark.sql.functions import udf, lit, explode
from pyspark import StorageLevel
import seaborn as sns
from sedona.utils.adapter import Adapter
from sedona.core.spatialOperator import KNNQuery
from shapely.geometry import Point
import pyspark.sql.functions as f
import numpy as np


import os
from IPython.display import display, HTML
from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.sql.types import LongType, BooleanType
from pyspark.sql.types import IntegerType, DoubleType, ArrayType
from pyspark.sql.functions import regexp_replace,pandas_udf,PandasUDFType
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from pyspark.sql.functions import col, split, expr
from pyspark.sql.functions import udf, lit, collect_list, collect_set
import findspark
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from pyspark.sql.functions import col, split, expr
from pyspark.sql.functions import udf, lit, explode
from pyspark import StorageLevel
import seaborn as sns
from sedona.utils.adapter import Adapter
from sedona.core.spatialOperator import KNNQuery
from shapely.geometry import Point
import pyspark.sql.functions as f
import torch
from torch import optim, nn
from torchvision import models, transforms
from PIL import Image
import numpy as np
from cv2 import cv2
from skimage.feature import *
from skimage.feature import greycoprops as gc
from pyspark.ml.feature import QuantileDiscretizer, Bucketizer
from pyspark.ml.feature import OneHotEncoder



In [3]:
spark = SparkSession.\
    builder.\
    master("spark://EN4119508L.cidse.dhcp.asu.edu:7077").\
    appName("euro_sat").\
    config("spark.serializer", KryoSerializer.getName).\
    config("spark.kryo.registrator", SedonaKryoRegistrator.getName) .\
    config("spark.driver.memory", "10g") .\
    config("spark.executor.memory", "64g") .\
    config("spark.driver.maxResultSize", "5g") .\
    config("spark.network.timeout", "1000s") .\
    config("spark.kryoserializer.buffer.max", "1024") .\
    config("spark.sql.broadcastTimeout", "36000") .\
    config("spark.sql.crossJoin.enabled", "true") .\
    getOrCreate()
SedonaRegistrator.registerAll(spark)
sc = spark.sparkContext


22/04/27 17:17:59 WARN Utils: Your hostname, en4119508l resolves to a loopback address: 127.0.1.1; using 10.218.106.130 instead (on interface enp0s25)
22/04/27 17:17:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/04/27 17:18:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/27 17:18:00 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/04/27 17:18:01 WARN Utils: The configured local directories are not expected to be URIs; however, got suspicious values [file:///hdd2/shantanuCodeData/lib/spark-logs]. Please check your configured local direc

In [9]:
GEOTIFF_DIR_BS = "../data2/euro_sat/water/"
train_bs = spark.read.format("geotiff").option("dropInvalid",True).load(GEOTIFF_DIR_BS)
water_df = train_bs.selectExpr("image.origin as origin","ST_GeomFromWkt(image.wkt) as Geom", "image.data as data", "image.nBands as bands")
water_df = water_df.selectExpr("origin", "Geom", "RS_Normalize(RS_GetBand(data,1, bands)) as aerosol", "RS_Normalize(Rs_GetBand(data, 2, bands)) as blue", "RS_Normalize(RS_GetBand(data, 3, bands)) as green", "RS_Normalize(Rs_GetBand(data, 4, bands)) as red", "RS_Normalize(RS_GetBand(data, 5, bands)) as red1", "RS_Normalize(RS_GetBand(data, 6, bands)) as red2", "RS_Normalize(RS_GetBand(data, 7, bands)) as red3", "RS_Normalize(RS_GetBand(data, 8, bands)) as nir", "RS_Normalize(RS_GetBand(data, 9, bands)) as red4", "RS_Normalize(RS_GetBand(data, 10, bands)) as vapor", "RS_Normalize(RS_GetBand(data, 11, bands)) as cirius", "RS_Normalize(RS_GetBand(data, 12, bands)) as swir_1", "RS_Normalize(RS_GetBand(data, 13, bands)) as swir_2")
water_df.show(2)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|              origin|                Geom|             aerosol|                blue|               green|                 red|                red1|                red2|                red3|                 nir|                red4|               vapor|              cirius|              swir_1|              swir_2|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|file:///hdd2/shan...|POLYGON ((-0.3627...|[247.0

In [10]:
GEOTIFF_DIR_BS = "../data2/euro_sat/land/"
train_bs = spark.read.format("geotiff").option("dropInvalid",True).load(GEOTIFF_DIR_BS)
land_df = train_bs.selectExpr("image.origin as origin","ST_GeomFromWkt(image.wkt) as Geom", "image.data as data", "image.nBands as bands")
land_df = land_df.selectExpr("origin", "Geom", "RS_Normalize(RS_GetBand(data,1, bands)) as aerosol", "RS_Normalize(Rs_GetBand(data, 2, bands)) as blue", "RS_Normalize(RS_GetBand(data, 3, bands)) as green", "RS_Normalize(Rs_GetBand(data, 4, bands)) as red", "RS_Normalize(RS_GetBand(data, 5, bands)) as red1", "RS_Normalize(RS_GetBand(data, 6, bands)) as red2", "RS_Normalize(RS_GetBand(data, 7, bands)) as red3", "RS_Normalize(RS_GetBand(data, 8, bands)) as nir", "RS_Normalize(RS_GetBand(data, 9, bands)) as red4", "RS_Normalize(RS_GetBand(data, 10, bands)) as vapor", "RS_Normalize(RS_GetBand(data, 11, bands)) as cirius", "RS_Normalize(RS_GetBand(data, 12, bands)) as swir_1", "RS_Normalize(RS_GetBand(data, 13, bands)) as swir_2")
land_df.show(2)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|              origin|                Geom|             aerosol|                blue|               green|                 red|                red1|                red2|                red3|                 nir|                red4|               vapor|              cirius|              swir_1|              swir_2|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|file:///hdd2/shan...|POLYGON ((18.0136...|[202.0

In [20]:
import time

GEOTIFF_DIR_BS = "../data2/euro_sat/load_data/"
train_bs = spark.read.format("geotiff").option("dropInvalid",True).load(GEOTIFF_DIR_BS)
train_df = train_bs.selectExpr("image.origin as origin","ST_GeomFromWkt(image.wkt) as Geom", "image.data as data", "image.nBands as bands")
train_df = train_df.selectExpr("origin", "Geom", "RS_Normalize(RS_GetBand(data,1, bands)) as aerosol", "RS_Normalize(Rs_GetBand(data, 2, bands)) as blue", "RS_Normalize(RS_GetBand(data, 3, bands)) as green", "RS_Normalize(Rs_GetBand(data, 4, bands)) as red", "RS_Normalize(RS_GetBand(data, 5, bands)) as red1", "RS_Normalize(RS_GetBand(data, 6, bands)) as red2", "RS_Normalize(RS_GetBand(data, 7, bands)) as red3", "RS_Normalize(RS_GetBand(data, 8, bands)) as nir", "RS_Normalize(RS_GetBand(data, 9, bands)) as red4", "RS_Normalize(RS_GetBand(data, 10, bands)) as vapor", "RS_Normalize(RS_GetBand(data, 11, bands)) as cirius", "RS_Normalize(RS_GetBand(data, 12, bands)) as swir_1", "RS_Normalize(RS_GetBand(data, 13, bands)) as swir_2")
t1 = time.time()
train_df.count()
t2 = time.time()

print(t2-t1)

62.26897144317627


In [22]:
from pyspark.sql.functions import lit

water_df = water_df.withColumn('class', lit(0))
land_df = land_df.withColumn('class', lit(1))

train_df = water_df.union(land_df)
train_df.persist().count()

19500

In [23]:
#spectral indices
import random

def ndwi(green, nir):
    result = []
    for i, r in enumerate(green):
        if r + nir[i] == 0:
            wi = random.uniform(-1, 1)
        else:
            wi = (r - nir[i])/(r + nir[i])
        result.append(wi)
    return result

def mndwi(green, swir):
    result = []
    for i, r in enumerate(green):
        wi = (r - swir[i])/(r + swir[i])
        result.append(wi)
    return result


def ndmi(nir, swir):
    result = []
    for i, r in enumerate(nir):
        wi = (r - swir[i])/(r + swir[i])
        result.append(wi)
    return result

def ndvi(nir, red):
    result = []
    for i, r in enumerate(nir):
        if r + red[i] == 0:
            wi = random.uniform(-1,1)
        else:
            wi = (r - red[i])/(r + red[i])
        result.append(wi)
    return result
# 4 * (Green - SWIR2) - (0.25 * NIR + 2.75 * SWIR1)


def awei(green, swir2, nir, swir1):
    result = []
    for i, r in enumerate(green):
        result.append(4*(r - swir2[i]) - (0.25*nir[i] + 2.75*swir1[i]))
        
    return result

def builtup_index(swir, nir):
    result = []
    for i, r in enumerate(swir):
        if r + nir[i] == 0:
            result.append(random.uniform(-1, 1))
        else:
            result.append((r - nir[i])/(r + nir[i]))
    return result



def rvi(nir, red):
    result = []
    for i, r in enumerate(nir):
        if red[i] == 0:
            result.append(random.uniform(-1, 1))
        else:
            result.append(r/red[i])
    return result

def average(data):
    return sum(data)/len(data)
   

spark.udf.register("ST_NDWI", ndwi, ArrayType(DoubleType()))
spark.udf.register("ST_MNDWI", mndwi, ArrayType(DoubleType()))
spark.udf.register("ST_NDMI", ndmi, ArrayType(DoubleType()))
spark.udf.register("ST_NDVI", ndvi, ArrayType(DoubleType()))
spark.udf.register("ST_AWEI", awei, ArrayType(DoubleType()))
spark.udf.register("ST_BI", builtup_index, ArrayType(DoubleType()))
spark.udf.register("ST_RVI", rvi, ArrayType(DoubleType()))
spark.udf.register("ST_Average", average, DoubleType())

df_index = train_df.selectExpr("origin", "ST_Average(ST_NDWI(green, nir)) as ndwi", "ST_Average(ST_MNDWI(green, swir_1)) as mdwi", "ST_Average(ST_NDMI(nir, swir_1)) as ndmi", "ST_Average(ST_NDVI(nir, red)) as ndvi", "ST_Average(ST_AWEI(green, swir_2, nir, swir_1)) as awei", "ST_Average(ST_BI(swir_1, nir)) as bi", "ST_Average(ST_RVI(nir, red)) as rvi")
df_index.count()        
    
    

22/04/27 17:47:07 WARN SimpleFunctionRegistry: The function st_ndwi replaced a previously registered function.
22/04/27 17:47:07 WARN SimpleFunctionRegistry: The function st_mndwi replaced a previously registered function.
22/04/27 17:47:07 WARN SimpleFunctionRegistry: The function st_ndmi replaced a previously registered function.
22/04/27 17:47:07 WARN SimpleFunctionRegistry: The function st_ndvi replaced a previously registered function.
22/04/27 17:47:07 WARN SimpleFunctionRegistry: The function st_awei replaced a previously registered function.
22/04/27 17:47:07 WARN SimpleFunctionRegistry: The function st_bi replaced a previously registered function.
22/04/27 17:47:07 WARN SimpleFunctionRegistry: The function st_rvi replaced a previously registered function.
22/04/27 17:47:07 WARN SimpleFunctionRegistry: The function st_average replaced a previously registered function.


19500

In [24]:
''' textural features from last dataset  '''

def rgbtograyscale(red, green, blue):
    a = (0.299*np.array(red) + 0.587*np.array(green) + 0.114*np.array(blue)).tolist()
    b = [round(x) for x in a]
    return b

def GLCM_Contrast(pixels):
    mi, ma = 0, 255
    ks = 5
    h,w = 64,64
    nbit = 8
    bins = np.linspace(mi, ma+1, nbit+1)
    bin_image = np.digitize(pixels, bins) - 1
    new_image = np.reshape(bin_image, (64, 64))
    glcm = greycomatrix(new_image, [1], [0, np.pi/4, np.pi/2], levels=8 , normed=True, symmetric=True)
    feature = gc(glcm, "contrast")
    return sum(feature[0].tolist())/len(feature[0].tolist())

def GLCM_Dissimilarity(pixels):
    mi, ma = 0, 255
    ks = 5
    h,w = 64,64
    nbit = 8
    bins = np.linspace(mi, ma+1, nbit+1)
    bin_image = np.digitize(pixels, bins) - 1
    new_image = np.reshape(bin_image, (64,64))
    glcm = greycomatrix(new_image, [1], [0, np.pi/4, np.pi/2], levels=8 , normed=True, symmetric=True)
    feature = gc(glcm, "dissimilarity")
    return sum(feature[0].tolist())/len(feature[0].tolist())


def GLCM_Homogeneity(pixels):
    mi, ma = 0, 255
    ks = 5
    h,w = 64,64
    nbit = 8
    bins = np.linspace(mi, ma+1, nbit+1)
    bin_image = np.digitize(pixels, bins) - 1
    new_image = np.reshape(bin_image, (64,64))
    glcm = greycomatrix(new_image, [1], [0, np.pi/4, np.pi/2], levels=8 , normed=True, symmetric=True)
    feature = gc(glcm, "homogeneity")
    return sum(feature[0].tolist())/len(feature[0].tolist())

def GLCM_Energy(pixels):
    mi, ma = 0, 255
    ks = 5
    h,w = 64,64
    nbit = 8
    bins = np.linspace(mi, ma+1, nbit+1)
    bin_image = np.digitize(pixels, bins) - 1
    new_image = np.reshape(bin_image, (64,64))
    glcm = greycomatrix(new_image, [1], [0, np.pi/4, np.pi/2], levels=8 , normed=True, symmetric=True)
    feature = gc(glcm, "energy")
    return sum(feature[0].tolist())/len(feature[0].tolist())


def GLCM_Correlation(pixels):
    mi, ma = 0, 255
    ks = 5
    h,w = 64,64
    nbit = 8
    bins = np.linspace(mi, ma+1, nbit+1)
    bin_image = np.digitize(pixels, bins) - 1
    new_image = np.reshape(bin_image, (64,64))
    glcm = greycomatrix(new_image, [1], [0, np.pi/4, np.pi/2], levels=8 , normed=True, symmetric=True)
    feature = gc(glcm, "correlation")
    return sum(feature[0].tolist())/len(feature[0].tolist())

def GLCM_ASM(pixels):
    mi, ma = 0, 255
    ks = 5
    h,w = 64,64
    nbit = 8
    bins = np.linspace(mi, ma+1, nbit+1)
    bin_image = np.digitize(pixels, bins) - 1
    new_image = np.reshape(bin_image, (64,64))
    glcm = greycomatrix(new_image, [1], [np.pi/4,np.pi/2], levels=8 , normed=True, symmetric=True)
    feature = gc(glcm, "ASM")
    return sum(feature[0].tolist())/len(feature[0].tolist())


spark.udf.register("RS_Convert", rgbtograyscale, ArrayType(IntegerType()))
spark.udf.register("GLCM_Contrast", GLCM_Contrast,DoubleType())
spark.udf.register("GLCM_Dis", GLCM_Dissimilarity, DoubleType())
spark.udf.register("GLCM_Hom", GLCM_Homogeneity, DoubleType())
spark.udf.register("GLCM_ASM", GLCM_ASM, DoubleType())
spark.udf.register("GLCM_Corr", GLCM_Correlation, DoubleType())
spark.udf.register("GLCM_Energy", GLCM_Energy, DoubleType())

gray_scale_dF = train_df.selectExpr("origin", "RS_Convert(red, green, blue) as gray_scale", "class")
text_df = gray_scale_dF.selectExpr("origin", "GLCM_Contrast(gray_scale) as glcm_contrast","GLCM_Dis(gray_scale) as glcm_dissimilarity", "GLCM_Hom(gray_scale) as glcm_homogeneity", "GLCM_Energy(gray_scale) as glcm_energy", "GLCM_Corr(gray_scale) as glcm_correlation","GLCM_ASM(gray_scale) as glcm_ASM", "class")
text_df.count()

22/04/27 17:47:22 WARN SimpleFunctionRegistry: The function rs_convert replaced a previously registered function.
22/04/27 17:47:22 WARN SimpleFunctionRegistry: The function glcm_contrast replaced a previously registered function.
22/04/27 17:47:22 WARN SimpleFunctionRegistry: The function glcm_dis replaced a previously registered function.
22/04/27 17:47:22 WARN SimpleFunctionRegistry: The function glcm_hom replaced a previously registered function.
22/04/27 17:47:22 WARN SimpleFunctionRegistry: The function glcm_asm replaced a previously registered function.
22/04/27 17:47:22 WARN SimpleFunctionRegistry: The function glcm_corr replaced a previously registered function.
22/04/27 17:47:22 WARN SimpleFunctionRegistry: The function glcm_energy replaced a previously registered function.


19500

In [18]:
indices_df = df_index.toPandas()
textural_df = text_df.toPandas()




In [34]:
final_textural_df = indices_df.merge(textural_df, on="origin", how="inner")
final_textural_df.to_csv("../data2/euro_sat/textural_features/textural.csv", index=False)

In [3]:
DATA_DIR = "/hdd2/shantanuCodeData/data/pbf/euro_sat/"
nodes = spark.read.parquet(DATA_DIR + "europe-latest.osm.pbf.node.parquet")
nodes = nodes.selectExpr("id","tags","CONCAT(longitude, ',',  latitude) as geomText")
nodes = nodes.select("id","geomText","tags")
nodes.persist().show()

def getlen(col):
    return len(col)

def regUDF(spark):
    length = udf(getlen, IntegerType())
    spark.udf.register("ColLen", length)

def constructGeometry(col):
    temp = []
    col = sorted(col, key=lambda x: int(x[0]))
    first_geom = col[0][1]
    for arr in col:
        temp.append(arr[1])
    temp.append(first_geom)
    return ",".join(temp)
    

def reggeomUDF(spark):
    geom = udf(constructGeometry, StringType())
    spark.udf.register("GetGeom", geom)

ways = spark.read.parquet(DATA_DIR + "europe-latest.osm.pbf.way.parquet")
ways = ways.select("id","tags",explode("nodes")).select(
  "*", col("col")["index"].alias("index"), col("col")["nodeId"].alias("nodeId")
    )
ways.createOrReplaceTempView("ways")
nodes.createOrReplaceTempView("points")
waysJoinnodes = spark.sql("Select ways.id as waysId,points.id as nodeId,ways.index as nodeindex, ways.tags as tags,points.geomText as geomText from points JOIN ways ON points.id=ways.nodeId")
waysJoinnodes = waysJoinnodes.withColumn("index_geom", f.array(f.col("nodeindex"), f.col("geomText"))).select("waysId", "nodeId", "tags", "index_geom")
waysJoinnodes = waysJoinnodes.groupBy("waysId").agg(collect_set("index_geom").alias('array_geom'))
regUDF(spark)
reggeomUDF(spark)
waysJoinnodes = waysJoinnodes.selectExpr("waysId", "array_geom","ColLen(array_geom) as length").where("length>=3").selectExpr("waysId", "array_geom", "GetGeom(array_geom) as Geometry")
waysJoinnodes.persist().show()
waysJoinnodes.createOrReplaceTempView("waysjoin")
finalways = spark.sql("Select ways.id as id, ways.tags as tags, waysjoin.Geometry as Geometry from ways JOIN waysjoin ON ways.id=waysjoin.waysId ")
finalways =  finalways.distinct().select("id", explode("tags"), "Geometry")
nodes = nodes.select("id","geomText",explode("tags")).selectExpr("id","ST_PointFromText(geomtext, ',') as Geometry", "col as attribute")
nodes.createOrReplaceTempView("nodes")
ways = finalways.selectExpr("id","ST_PolygonFromText(Geometry, ',') as Geometry", "col as attribute")
desiredCoordinates = nodes.select(
  "*", col("attribute")["key"].alias("key"), col("attribute")["value"].alias("value"))
ways = ways.select(
  "*", col("attribute")["key"].alias("key"), col("attribute")["value"].alias("value"))
def getString(hexa):
    return hexa.decode()
convert = udf(getString, StringType())
spark.udf.register("RS_Convert", convert)
points = desiredCoordinates.selectExpr("id", "Geometry","RS_Convert(key) as attr_key","RS_Convert(value) as attr_value")
polygons = ways.selectExpr("id", "Geometry","RS_Convert(key) as attr_key","RS_Convert(value) as attr_value")
# desiredCoordinates.unpersist()
# points.persist().show(2)
polygons.persist().show(2)


+---+--------------------+--------------------+
| id|            geomText|                tags|
+---+--------------------+--------------------+
|  1|13.56900320000000...|[{communication:m...|
|  2|30.32611000000000...|[{historic, memor...|
| 69|12.86222370000000...|   [{natural, tree}]|
|100|10.8340913,52.891...|[{description, We...|
|110|10.78341500000000...|                  []|
|111|10.7875238,59.947...|                  []|
|112|10.7854259,59.951...|[{barrier, block}...|
|113|10.78193170000000...|                  []|
|114|10.78433900000000...|                  []|
|115|10.7796921,59.951...|                  []|
|116|10.7781722,59.952...|                  []|
|117|10.78134160000000...|                  []|
|118|10.7856298,59.951...|                  []|
|119|10.7862779,59.951...|                  []|
|120|10.78726150000000...|                  []|
|121|10.7892711,59.950...|                  []|
|122|10.7898343,59.950...|                  []|
|123|10.7899198,59.950...|              

+-------+--------------------+--------------------+
| waysId|          array_geom|            Geometry|
+-------+--------------------+--------------------+
|    474|[[1, -0.805012800...|-0.80630750000000...|
|    964|[[0, -1.798361900...|-1.79836190000000...|
|   1677|[[1, -1.3948659,5...|-1.3941614,50.926...|
|   2214|[[0, -0.0166724,5...|-0.0166724,51.426...|
|   2509|[[3, -1.834021000...|-1.8339437,52.650...|
|   2529|[[5, -2.3599828,5...|-2.35976750000000...|
|   2927|[[10, -0.03121260...|-0.02936080000000...|
|  83646|[[17, -4.79418370...|-4.78877960000000...|
| 252993|[[0, -1.372270900...|-1.37227090000000...|
| 260048|[[1, -0.492700200...|-0.4929179,51.370...|
| 276858|[[4, -0.1928493,5...|-0.1948544,51.602...|
| 314372|[[3, 8.4701543,55...|8.4702433,55.4762...|
| 363447|[[2, 18.0744036,5...|18.0742134,59.340...|
| 374036|[[1, 16.2739489,4...|16.27396790000000...|
| 599927|[[6, -1.5417018,5...|-1.5413977,50.676...|
| 838032|[[4, -2.841060700...|-2.8411778,53.185...|
| 959021|[[5

+---+--------------------+-----------+---------------+
| id|            Geometry|   attr_key|     attr_value|
+---+--------------------+-----------+---------------+
|474|POLYGON ((-0.8063...|designation|public_footpath|
|474|POLYGON ((-0.8063...|       foot|            yes|
+---+--------------------+-----------+---------------+
only showing top 2 rows



In [4]:
# points.write.parquet("/hdd2/shantanuCodeData/data2/euro_sat/kb/nodes.parquet")
polygons.write.parquet("/hdd2/shantanuCodeData/data2/euro_sat/kb/ways.parquet")


# Geospatial features using OSM knowledge vase of europe 

In [25]:
def get_origin(text):
    return text.split("/")[-1]
spark.udf.register("ST_Origin", get_origin, StringType())


geo_train_df = train_df.selectExpr("ST_Origin(origin) as origin", "Geom", "class")
geo_train_df.show()


+----------------+--------------------+-----+
|          origin|                Geom|class|
+----------------+--------------------+-----+
|SeaLake_2847.tif|POLYGON ((-0.3627...|    0|
| SeaLake_363.tif|POLYGON ((12.2137...|    0|
|  River_2019.tif|POLYGON ((9.27551...|    0|
|SeaLake_2111.tif|POLYGON ((8.32115...|    0|
| SeaLake_734.tif|POLYGON ((5.21409...|    0|
| SeaLake_430.tif|POLYGON ((28.0818...|    0|
|SeaLake_1243.tif|POLYGON ((14.1621...|    0|
| SeaLake_709.tif|POLYGON ((4.33435...|    0|
|SeaLake_1891.tif|POLYGON ((0.64670...|    0|
|   River_495.tif|POLYGON ((0.80404...|    0|
|SeaLake_2832.tif|POLYGON ((-1.6822...|    0|
|SeaLake_2583.tif|POLYGON ((20.0949...|    0|
|SeaLake_2643.tif|POLYGON ((4.11489...|    0|
|  River_2095.tif|POLYGON ((9.58068...|    0|
|SeaLake_2582.tif|POLYGON ((25.4956...|    0|
|SeaLake_2657.tif|POLYGON ((10.4027...|    0|
|SeaLake_2834.tif|POLYGON ((20.9071...|    0|
|SeaLake_2317.tif|POLYGON ((14.3050...|    0|
|  River_2032.tif|POLYGON ((4.8796

In [26]:
points = spark.read.parquet("../data2/euro_sat/kb/nodes.parquet")
polygons = spark.read.parquet("../data2/euro_sat/kb/ways.parquet")

joint_kb = points.union(polygons)

forest_points = points.filter((points.attr_key=="landuse") & (points.attr_value=="forest"))
forest_polygons = polygons.filter((polygons.attr_key=="landuse") & (polygons.attr_value=="forest"))

forest_kb = forest_points.union(forest_polygons)
forest_kb.persist().show()

+----------+--------------------+--------+----------+
|        id|            Geometry|attr_key|attr_value|
+----------+--------------------+--------+----------+
|3123477544|POINT (12.2705131...| landuse|    forest|
| 395384461|POINT (2.4340426 ...| landuse|    forest|
|4757465299|POINT (9.0373967 ...| landuse|    forest|
|4767431496|POINT (9.495564 4...| landuse|    forest|
|4767480875|POINT (9.4964887 ...| landuse|    forest|
|4768270166|POINT (9.4990301 ...| landuse|    forest|
|4768270167|POINT (9.4992085 ...| landuse|    forest|
| 295876412|POINT (11.0076779...| landuse|    forest|
| 295876414|POINT (10.9949740...| landuse|    forest|
| 295876415|POINT (11.0759655...| landuse|    forest|
| 295876416|POINT (11.0781206...| landuse|    forest|
| 295876949|POINT (11.0218181...| landuse|    forest|
| 470624533|POINT (9.2603281 ...| landuse|    forest|
| 470949988|POINT (13.8594593...| landuse|    forest|
| 470950419|POINT (13.9156899...| landuse|    forest|
| 471127707|POINT (10.217456

In [21]:
''' Spatial joins with sealakes, river, forest, vegeration, highway, industrial and residential data points '''
geo_train_df.show(2)
geo_train_df.createOrReplaceTempView("train")

forest_kb.createOrReplaceTempView("forest_kb")
forest_kb.createOrReplaceTempView("forest_kb")
forest_overlap = spark.sql("select origin, cast(ST_Overlaps(train.geom, forest_kb.Geometry) as INT) as isOverlaps_forest, class from train, forest_kb") 
forest_distance = spark.sql("select origin, ST_Distance(train.geom, forest_kb.Geometry) as distance_forest, class from train, forest_kb")
forest_intersects = spark.sql("select origin, cast(ST_Intersects(train.geom, forest_kb.Geometry) as INT) as isIntersect_forest, class from train, forest_kb") 
forest_contains = spark.sql("select origin, cast(ST_Contains(train.geom, forest_kb.Geometry) as INT) as isContain_forest, class from train, forest_kb") 
forest_touches = spark.sql("select origin, cast(ST_Touches(train.geom, forest_kb.Geometry) as INT) as isTouches_forest, class from train, forest_kb") 

forest_overlap.persist().show()
forest_distance.persist().show()
forest_intersects.persist().show()
forest_contains.persist().show()
forest_touches.persist().show()




+----------------+--------------------+-----+
|          origin|                Geom|class|
+----------------+--------------------+-----+
|SeaLake_2847.tif|POLYGON ((-0.3627...|    0|
| SeaLake_363.tif|POLYGON ((12.2137...|    0|
+----------------+--------------------+-----+
only showing top 2 rows

+----------------+-----------------+-----+
|          origin|isOverlaps_forest|class|
+----------------+-----------------+-----+
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_28

22/04/23 21:36:24 WARN CacheManager: Asked to cache already cached data.
22/04/23 21:36:24 WARN CacheManager: Asked to cache already cached data.


+----------------+------------------+-----+
|          origin|   distance_forest|class|
+----------------+------------------+-----+
|SeaLake_2847.tif| 15.92138426593651|    0|
| SeaLake_363.tif|6.9401393240031615|    0|
|  River_2019.tif|   6.0205898028452|    0|
|SeaLake_2111.tif| 5.231899424653846|    0|
| SeaLake_734.tif| 7.766154578466899|    0|
| SeaLake_430.tif|16.211476666124316|    0|
|SeaLake_1243.tif| 9.130463626835864|    0|
| SeaLake_709.tif| 7.963704188046001|    0|
|SeaLake_1891.tif|11.872130347015137|    0|
|   River_495.tif|11.463836166350218|    0|
|SeaLake_2832.tif|14.081808685514021|    0|
|SeaLake_2583.tif| 8.287736427012202|    0|
|SeaLake_2643.tif| 8.596076084428088|    0|
|  River_2095.tif|2.8234581237121774|    0|
|SeaLake_2582.tif|18.009620181670627|    0|
|SeaLake_2657.tif|5.5746050154751305|    0|
|SeaLake_2834.tif| 12.76047246395043|    0|
|SeaLake_2317.tif| 9.167645154537414|    0|
|  River_2032.tif|  7.71836638868871|    0|
|SeaLake_1930.tif| 4.48853734595

22/04/23 21:36:24 WARN CacheManager: Asked to cache already cached data.


+----------------+------------------+-----+
|          origin|isIntersect_forest|class|
+----------------+------------------+-----+
|SeaLake_2847.tif|                 0|    0|
| SeaLake_363.tif|                 0|    0|
|  River_2019.tif|                 0|    0|
|SeaLake_2111.tif|                 0|    0|
| SeaLake_734.tif|                 0|    0|
| SeaLake_430.tif|                 0|    0|
|SeaLake_1243.tif|                 0|    0|
| SeaLake_709.tif|                 0|    0|
|SeaLake_1891.tif|                 0|    0|
|   River_495.tif|                 0|    0|
|SeaLake_2832.tif|                 0|    0|
|SeaLake_2583.tif|                 0|    0|
|SeaLake_2643.tif|                 0|    0|
|  River_2095.tif|                 0|    0|
|SeaLake_2582.tif|                 0|    0|
|SeaLake_2657.tif|                 0|    0|
|SeaLake_2834.tif|                 0|    0|
|SeaLake_2317.tif|                 0|    0|
|  River_2032.tif|                 0|    0|
|SeaLake_1930.tif|              

22/04/23 21:36:24 WARN CacheManager: Asked to cache already cached data.


+----------------+----------------+-----+
|          origin|isContain_forest|class|
+----------------+----------------+-----+
|SeaLake_2847.tif|               0|    0|
| SeaLake_363.tif|               0|    0|
|  River_2019.tif|               0|    0|
|SeaLake_2111.tif|               0|    0|
| SeaLake_734.tif|               0|    0|
| SeaLake_430.tif|               0|    0|
|SeaLake_1243.tif|               0|    0|
| SeaLake_709.tif|               0|    0|
|SeaLake_1891.tif|               0|    0|
|   River_495.tif|               0|    0|
|SeaLake_2832.tif|               0|    0|
|SeaLake_2583.tif|               0|    0|
|SeaLake_2643.tif|               0|    0|
|  River_2095.tif|               0|    0|
|SeaLake_2582.tif|               0|    0|
|SeaLake_2657.tif|               0|    0|
|SeaLake_2834.tif|               0|    0|
|SeaLake_2317.tif|               0|    0|
|  River_2032.tif|               0|    0|
|SeaLake_1930.tif|               0|    0|
+----------------+----------------

22/04/23 21:36:25 WARN CacheManager: Asked to cache already cached data.


+----------------+----------------+-----+
|          origin|isTouches_forest|class|
+----------------+----------------+-----+
|SeaLake_2847.tif|               0|    0|
| SeaLake_363.tif|               0|    0|
|  River_2019.tif|               0|    0|
|SeaLake_2111.tif|               0|    0|
| SeaLake_734.tif|               0|    0|
| SeaLake_430.tif|               0|    0|
|SeaLake_1243.tif|               0|    0|
| SeaLake_709.tif|               0|    0|
|SeaLake_1891.tif|               0|    0|
|   River_495.tif|               0|    0|
|SeaLake_2832.tif|               0|    0|
|SeaLake_2583.tif|               0|    0|
|SeaLake_2643.tif|               0|    0|
|  River_2095.tif|               0|    0|
|SeaLake_2582.tif|               0|    0|
|SeaLake_2657.tif|               0|    0|
|SeaLake_2834.tif|               0|    0|
|SeaLake_2317.tif|               0|    0|
|  River_2032.tif|               0|    0|
|SeaLake_1930.tif|               0|    0|
+----------------+----------------

In [25]:
df_1 = forest_overlap.groupby("origin").sum("isOverlaps_forest").alias("total_overlap_forest")
df_2 = forest_distance.groupby("origin").min("distance_forest").alias("distance_forest")
df_3 = forest_contains.groupby("origin").sum("isContain_forest").alias("total_contain_forest")
df_4 = forest_touches.groupby("origin").sum("isTouches_forest").alias("total_touch_forest")
df_5 = forest_intersects.groupby("origin").sum("isIntersect_forest").alias("total_intersect_forest")

df_1.persist().count()
df_2.persist().count()
df_3.persist().count()
df_4.persist().count()
df_5.persist().count()

19500

In [27]:
df1 = df_1.toPandas()
df2 = df_2.toPandas()
df3 = df_3.toPandas()
df4 = df_4.toPandas()
df5 = df_5.toPandas()

In [42]:
df_final = df1.merge(df2, on='origin', how="inner")
df_final = df3.merge(df_final, on="origin", how="inner")
df_final = df4.merge(df_final, on="origin", how="inner")
df_final = df5.merge(df_final, on="origin", how="inner")

df_final.to_csv("../data2/euro_sat/forest.csv", index=False)

In [11]:
industrial_points = points.filter((points.attr_key=="landuse") & (points.attr_value=="industrial"))
industrial_polygons = polygons.filter((polygons.attr_key=="landuse") & (polygons.attr_value=="industrial"))

industrial_kb = industrial_points.union(industrial_polygons)
industrial_kb.persist().show()
industrial_kb.createOrReplaceTempView("industry_kb")
industrial_overlap = spark.sql("select origin, cast(ST_Overlaps(train.geom, industry_kb.Geometry) as INT) as isOverlaps_industry, class from train, industry_kb") 
industrial_distance = spark.sql("select origin, ST_Distance(train.geom, industry_kb.Geometry) as distance_industry, class from train, industry_kb")
industrial_contains = spark.sql("select origin, cast(ST_Contains(train.geom, industry_kb.Geometry) as INT) as isContain_industry, class from train, industry_kb") 
industrial_intersects = spark.sql("select origin, cast(ST_Intersects(train.geom, industry_kb.Geometry) as INT) as isIntersect_industry, class from train, industry_kb") 
industrial_touches = spark.sql("select origin, cast(ST_Touches(train.geom, industry_kb.Geometry) as INT) as isTouch_industry, class from train, industry_kb") 
industrial_overlap.persist().show()
industrial_distance.persist().show()
industrial_contains.persist().show()
industrial_intersects.persist().show()
industrial_touches.persist().show()



+----------+--------------------+--------+----------+
|        id|            Geometry|attr_key|attr_value|
+----------+--------------------+--------+----------+
| 343120423|POINT (-1.1806796...| landuse|industrial|
| 966347483|POINT (20.8315943...| landuse|industrial|
|3122745778|POINT (11.1305472...| landuse|industrial|
|3122785289|POINT (11.4080631...| landuse|industrial|
|3131028444|POINT (-1.4895529...| landuse|industrial|
|3134432145|POINT (16.58014 5...| landuse|industrial|
| 385374112|POINT (21.6291288...| landuse|industrial|
| 385592207|POINT (-5.8403852...| landuse|industrial|
| 386940286|POINT (-2.3005779...| landuse|industrial|
| 392105051|POINT (7.1700663 ...| landuse|industrial|
|4748573723|POINT (11.7685078...| landuse|industrial|
|4748663370|POINT (32.2659704...| landuse|industrial|
| 289094210|POINT (0.6890868 ...| landuse|industrial|
| 289094211|POINT (0.6872954 ...| landuse|industrial|
| 289094212|POINT (0.685662 5...| landuse|industrial|
| 289094213|POINT (0.6867909

+----------------+-------------------+-----+
|          origin|isOverlaps_industry|class|
+----------------+-------------------+-----+
|SeaLake_2847.tif|                  0|    0|
|SeaLake_2847.tif|                  0|    0|
|SeaLake_2847.tif|                  0|    0|
|SeaLake_2847.tif|                  0|    0|
|SeaLake_2847.tif|                  0|    0|
|SeaLake_2847.tif|                  0|    0|
|SeaLake_2847.tif|                  0|    0|
|SeaLake_2847.tif|                  0|    0|
|SeaLake_2847.tif|                  0|    0|
|SeaLake_2847.tif|                  0|    0|
|SeaLake_2847.tif|                  0|    0|
|SeaLake_2847.tif|                  0|    0|
|SeaLake_2847.tif|                  0|    0|
|SeaLake_2847.tif|                  0|    0|
|SeaLake_2847.tif|                  0|    0|
|SeaLake_2847.tif|                  0|    0|
|SeaLake_2847.tif|                  0|    0|
|SeaLake_2847.tif|                  0|    0|
|SeaLake_2847.tif|                  0|    0|
|SeaLake_2

+----------------+------------------+-----+
|          origin| distance_industry|class|
+----------------+------------------+-----+
|SeaLake_2847.tif| 13.62697702091927|    0|
|SeaLake_2847.tif| 21.25205817573253|    0|
|SeaLake_2847.tif|23.269014440354436|    0|
|SeaLake_2847.tif|23.363002678933412|    0|
|SeaLake_2847.tif|14.242571139149645|    0|
|SeaLake_2847.tif| 21.50685936722675|    0|
|SeaLake_2847.tif|22.997011016253968|    0|
|SeaLake_2847.tif|6.7869513399955546|    0|
|SeaLake_2847.tif|12.540518801623419|    0|
|SeaLake_2847.tif|14.654297186708982|    0|
|SeaLake_2847.tif|13.063282779659225|    0|
|SeaLake_2847.tif|33.883840560389665|    0|
|SeaLake_2847.tif| 12.93463479737444|    0|
|SeaLake_2847.tif|12.933956492719917|    0|
|SeaLake_2847.tif|12.933876335801349|    0|
|SeaLake_2847.tif|12.935325926928197|    0|
|SeaLake_2847.tif|12.937544078153012|    0|
|SeaLake_2847.tif|12.938038852678567|    0|
|SeaLake_2847.tif| 12.93683417403714|    0|
|SeaLake_2847.tif|12.93668997756

+----------------+------------------+-----+
|          origin|isContain_industry|class|
+----------------+------------------+-----+
|SeaLake_2847.tif|                 0|    0|
|SeaLake_2847.tif|                 0|    0|
|SeaLake_2847.tif|                 0|    0|
|SeaLake_2847.tif|                 0|    0|
|SeaLake_2847.tif|                 0|    0|
|SeaLake_2847.tif|                 0|    0|
|SeaLake_2847.tif|                 0|    0|
|SeaLake_2847.tif|                 0|    0|
|SeaLake_2847.tif|                 0|    0|
|SeaLake_2847.tif|                 0|    0|
|SeaLake_2847.tif|                 0|    0|
|SeaLake_2847.tif|                 0|    0|
|SeaLake_2847.tif|                 0|    0|
|SeaLake_2847.tif|                 0|    0|
|SeaLake_2847.tif|                 0|    0|
|SeaLake_2847.tif|                 0|    0|
|SeaLake_2847.tif|                 0|    0|
|SeaLake_2847.tif|                 0|    0|
|SeaLake_2847.tif|                 0|    0|
|SeaLake_2847.tif|              

+----------------+--------------------+-----+
|          origin|isIntersect_industry|class|
+----------------+--------------------+-----+
|SeaLake_2847.tif|                   0|    0|
|SeaLake_2847.tif|                   0|    0|
|SeaLake_2847.tif|                   0|    0|
|SeaLake_2847.tif|                   0|    0|
|SeaLake_2847.tif|                   0|    0|
|SeaLake_2847.tif|                   0|    0|
|SeaLake_2847.tif|                   0|    0|
|SeaLake_2847.tif|                   0|    0|
|SeaLake_2847.tif|                   0|    0|
|SeaLake_2847.tif|                   0|    0|
|SeaLake_2847.tif|                   0|    0|
|SeaLake_2847.tif|                   0|    0|
|SeaLake_2847.tif|                   0|    0|
|SeaLake_2847.tif|                   0|    0|
|SeaLake_2847.tif|                   0|    0|
|SeaLake_2847.tif|                   0|    0|
|SeaLake_2847.tif|                   0|    0|
|SeaLake_2847.tif|                   0|    0|
|SeaLake_2847.tif|                

+----------------+----------------+-----+
|          origin|isTouch_industry|class|
+----------------+----------------+-----+
|SeaLake_2847.tif|               0|    0|
|SeaLake_2847.tif|               0|    0|
|SeaLake_2847.tif|               0|    0|
|SeaLake_2847.tif|               0|    0|
|SeaLake_2847.tif|               0|    0|
|SeaLake_2847.tif|               0|    0|
|SeaLake_2847.tif|               0|    0|
|SeaLake_2847.tif|               0|    0|
|SeaLake_2847.tif|               0|    0|
|SeaLake_2847.tif|               0|    0|
|SeaLake_2847.tif|               0|    0|
|SeaLake_2847.tif|               0|    0|
|SeaLake_2847.tif|               0|    0|
|SeaLake_2847.tif|               0|    0|
|SeaLake_2847.tif|               0|    0|
|SeaLake_2847.tif|               0|    0|
|SeaLake_2847.tif|               0|    0|
|SeaLake_2847.tif|               0|    0|
|SeaLake_2847.tif|               0|    0|
|SeaLake_2847.tif|               0|    0|
+----------------+----------------

In [31]:
df_6 = industrial_overlap.groupby("origin").sum("isOverlaps_industry").alias("total_overlap_ind")
df_7 = industrial_distance.groupby("origin").min("distance_industry").alias("distance_industrial")
df_8 = industrial_contains.groupby("origin").sum("isContain_industry").alias("total_contain_industrial")
df_9 = industrial_touches.groupby("origin").sum("isTouch_industry").alias("total_touch_industrial")
df_10 = industrial_intersects.groupby("origin").sum("isIntersect_industry").alias("total_intersect_industrial")

df_6.persist().count()
df_7.persist().count()
df_8.persist().count()
df_9.persist().count()
df_10.persist().count()



19500

In [32]:
df6 = df_6.toPandas()
df7 = df_7.toPandas()
df8 = df_8.toPandas()
df9 = df_9.toPandas()
df10 = df_10.toPandas()


In [43]:
df_final = df6.merge(df7, on='origin', how="inner")
df_final = df8.merge(df_final, on="origin", how="inner")
df_final = df9.merge(df_final, on="origin", how="inner")
df_final = df10.merge(df_final, on="origin", how="inner")

df_final.to_csv("../data2/euro_sat/industry.csv", index=False)

In [13]:
residential_points = points.filter((points.attr_key=="landuse") & (points.attr_value=="residential"))
residential_polygons = polygons.filter((polygons.attr_key=="landuse") & (polygons.attr_value=="residential"))

residential_kb = residential_points.union(residential_polygons)
residential_kb.persist().count()
residential_kb.createOrReplaceTempView("residential_kb")
residential_overlap = spark.sql("select origin, cast(ST_Overlaps(train.geom, residential_kb.Geometry) as INT) as isOverlaps_residential, class from train, residential_kb") 
residential_distance = spark.sql("select origin, ST_Distance(train.geom, residential_kb.Geometry) as distance_residential, class from train, residential_kb")
residential_contains = spark.sql("select origin, cast(ST_Contains(train.geom, residential_kb.Geometry) as INT) as isContain_residential, class from train, residential_kb") 
residential_intersects = spark.sql("select origin, cast(ST_Intersects(train.geom, residential_kb.Geometry) as INT) as isIntersect_residential, class from train, residential_kb") 
residential_touches = spark.sql("select origin, cast(ST_Touches(train.geom, residential_kb.Geometry) as INT) as isTouch_residential, class from train, residential_kb") 

residential_overlap.persist().show()
residential_distance.persist().show()
residential_contains.persist().show()
residential_intersects.persist().show()
residential_touches.persist().show()

22/04/23 20:45:24 WARN CacheManager: Asked to cache already cached data.
22/04/23 20:45:24 WARN CacheManager: Asked to cache already cached data.
22/04/23 20:45:24 WARN CacheManager: Asked to cache already cached data.


+----------------+----------------------+-----+
|          origin|isOverlaps_residential|class|
+----------------+----------------------+-----+
|SeaLake_2847.tif|                     0|    0|
|SeaLake_2847.tif|                     0|    0|
|SeaLake_2847.tif|                     0|    0|
| SeaLake_363.tif|                     0|    0|
| SeaLake_363.tif|                     0|    0|
| SeaLake_363.tif|                     0|    0|
|  River_2019.tif|                     0|    0|
|  River_2019.tif|                     0|    0|
|  River_2019.tif|                     0|    0|
|SeaLake_2111.tif|                     0|    0|
|SeaLake_2111.tif|                     0|    0|
|SeaLake_2111.tif|                     0|    0|
| SeaLake_734.tif|                     0|    0|
| SeaLake_734.tif|                     0|    0|
| SeaLake_734.tif|                     0|    0|
| SeaLake_430.tif|                     0|    0|
| SeaLake_430.tif|                     0|    0|
| SeaLake_430.tif|                     0

22/04/23 20:45:25 WARN CacheManager: Asked to cache already cached data.
22/04/23 20:45:25 WARN CacheManager: Asked to cache already cached data.


+----------------+---------------------+-----+
|          origin|isContain_residential|class|
+----------------+---------------------+-----+
|SeaLake_2847.tif|                    0|    0|
|SeaLake_2847.tif|                    0|    0|
|SeaLake_2847.tif|                    0|    0|
| SeaLake_363.tif|                    0|    0|
| SeaLake_363.tif|                    0|    0|
| SeaLake_363.tif|                    0|    0|
|  River_2019.tif|                    0|    0|
|  River_2019.tif|                    0|    0|
|  River_2019.tif|                    0|    0|
|SeaLake_2111.tif|                    0|    0|
|SeaLake_2111.tif|                    0|    0|
|SeaLake_2111.tif|                    0|    0|
| SeaLake_734.tif|                    0|    0|
| SeaLake_734.tif|                    0|    0|
| SeaLake_734.tif|                    0|    0|
| SeaLake_430.tif|                    0|    0|
| SeaLake_430.tif|                    0|    0|
| SeaLake_430.tif|                    0|    0|
|SeaLake_1243

In [33]:
df_11 = residential_overlap.groupby("origin").sum("isOverlaps_residential").alias("total_overlap_res")
df_12 = residential_distance.groupby("origin").min("distance_residential").alias("distance_res")
df_13 = residential_contains.groupby("origin").sum("isContain_residential").alias("total_contain_res")
df_14 = residential_touches.groupby("origin").sum("isTouch_residential").alias("total_touch_res")
df_15 = residential_intersects.groupby("origin").sum("isIntersect_residential").alias("total_intersect_res")

df_11.persist().count()
df_12.persist().count()
df_13.persist().count()
df_14.persist().count()
df_15.persist().count()

19500

In [34]:
df11 = df_11.toPandas()
df12 = df_12.toPandas()
df13 = df_13.toPandas()
df14 = df_14.toPandas()
df15 = df_15.toPandas()

In [44]:
df_final = df11.merge(df12, on='origin', how="inner")
df_final = df13.merge(df_final, on="origin", how="inner")
df_final = df14.merge(df_final, on="origin", how="inner")
df_final = df15.merge(df_final, on="origin", how="inner")

df_final.to_csv("../data2/euro_sat/residential.csv", index=False)

In [14]:
river_points = points.filter((points.attr_key=="waterway") & (points.attr_value=="river"))
river_polygons = polygons.filter((polygons.attr_key=="waterway") & (polygons.attr_value=="river"))
river_kb = river_points.union(river_polygons)
river_kb.show()
river_kb.persist().count()
river_kb.createOrReplaceTempView("river_kb")
river_overlap = spark.sql("select origin, cast(ST_Overlaps(train.geom, river_kb.Geometry) as INT) as isOverlaps_river, class from train, river_kb") 
river_distance = spark.sql("select origin, ST_Distance(train.geom, river_kb.Geometry) as distance_river, class from train, river_kb")
river_contains = spark.sql("select origin, cast(ST_Contains(train.geom, river_kb.Geometry) as INT) as isContain_river, class from train, river_kb") 
river_intersects = spark.sql("select origin, cast(ST_Intersects(train.geom, river_kb.Geometry) as INT) as isIntersect_river, class from train, river_kb") 
river_touches = spark.sql("select origin, cast(ST_Touches(train.geom, river_kb.Geometry) as INT) as isTouch_river, class from train, river_kb") 


river_distance.persist().show()
river_overlap.persist().show()
river_contains.persist().show()
river_intersects.persist().show()
river_touches.persist().show()



+----------+--------------------+--------+----------+
|        id|            Geometry|attr_key|attr_value|
+----------+--------------------+--------+----------+
|9075746305|POINT (10.2625583...|waterway|     river|
|9076734025|POINT (7.3714139 ...|waterway|     river|
|9077242944|POINT (6.90884170...|waterway|     river|
|9082818118|POINT (7.44840830...|waterway|     river|
|9089496939|POINT (7.66552780...|waterway|     river|
|9371327551|POINT (6.95857940...|waterway|     river|
|8341778501|POINT (23.6479049...|waterway|     river|
|8680223148|POINT (10.3615000...|waterway|     river|
|8682324704|POINT (7.2427611 ...|waterway|     river|
|8682399714|POINT (7.36202780...|waterway|     river|
|9350397741|POINT (10.4181462...|waterway|     river|
|6990009666|POINT (22.2643725...|waterway|     river|
|5196071552|POINT (9.3021887 ...|waterway|     river|
|5196071553|POINT (9.3126671 ...|waterway|     river|
|5196071554|POINT (9.3253622 ...|waterway|     river|
|5196071555|POINT (9.3384795

+----------------+------------------+-----+
|          origin|    distance_river|class|
+----------------+------------------+-----+
|SeaLake_2847.tif|23.677458834576523|    0|
| SeaLake_363.tif|18.507013383032085|    0|
|  River_2019.tif| 6.306190622671269|    0|
|SeaLake_2111.tif| 8.245580554305446|    0|
| SeaLake_734.tif| 9.620729313007882|    0|
| SeaLake_430.tif|23.320396475487858|    0|
|SeaLake_1243.tif| 4.646262117243028|    0|
| SeaLake_709.tif| 13.57902236628867|    0|
|SeaLake_1891.tif|13.162732231200275|    0|
|   River_495.tif|14.561146430614574|    0|
|SeaLake_2832.tif|17.965024420067888|    0|
|SeaLake_2583.tif|17.268646409078166|    0|
|SeaLake_2643.tif| 10.65994374997349|    0|
|  River_2095.tif|10.594522913017045|    0|
|SeaLake_2582.tif|15.252320723743619|    0|
|SeaLake_2657.tif| 6.203994300486309|    0|
|SeaLake_2834.tif|23.416680714000996|    0|
|SeaLake_2317.tif| 4.763208481398384|    0|
|  River_2032.tif|14.731214450718245|    0|
|SeaLake_1930.tif| 8.07375151182

+----------------+----------------+-----+
|          origin|isOverlaps_river|class|
+----------------+----------------+-----+
|SeaLake_2847.tif|               0|    0|
| SeaLake_363.tif|               0|    0|
|  River_2019.tif|               0|    0|
|SeaLake_2111.tif|               0|    0|
| SeaLake_734.tif|               0|    0|
| SeaLake_430.tif|               0|    0|
|SeaLake_1243.tif|               0|    0|
| SeaLake_709.tif|               0|    0|
|SeaLake_1891.tif|               0|    0|
|   River_495.tif|               0|    0|
|SeaLake_2832.tif|               0|    0|
|SeaLake_2583.tif|               0|    0|
|SeaLake_2643.tif|               0|    0|
|  River_2095.tif|               0|    0|
|SeaLake_2582.tif|               0|    0|
|SeaLake_2657.tif|               0|    0|
|SeaLake_2834.tif|               0|    0|
|SeaLake_2317.tif|               0|    0|
|  River_2032.tif|               0|    0|
|SeaLake_1930.tif|               0|    0|
+----------------+----------------

+----------------+---------------+-----+
|          origin|isContain_river|class|
+----------------+---------------+-----+
|SeaLake_2847.tif|              0|    0|
| SeaLake_363.tif|              0|    0|
|  River_2019.tif|              0|    0|
|SeaLake_2111.tif|              0|    0|
| SeaLake_734.tif|              0|    0|
| SeaLake_430.tif|              0|    0|
|SeaLake_1243.tif|              0|    0|
| SeaLake_709.tif|              0|    0|
|SeaLake_1891.tif|              0|    0|
|   River_495.tif|              0|    0|
|SeaLake_2832.tif|              0|    0|
|SeaLake_2583.tif|              0|    0|
|SeaLake_2643.tif|              0|    0|
|  River_2095.tif|              0|    0|
|SeaLake_2582.tif|              0|    0|
|SeaLake_2657.tif|              0|    0|
|SeaLake_2834.tif|              0|    0|
|SeaLake_2317.tif|              0|    0|
|  River_2032.tif|              0|    0|
|SeaLake_1930.tif|              0|    0|
+----------------+---------------+-----+
only showing top

+----------------+-----------------+-----+
|          origin|isIntersect_river|class|
+----------------+-----------------+-----+
|SeaLake_2847.tif|                0|    0|
| SeaLake_363.tif|                0|    0|
|  River_2019.tif|                0|    0|
|SeaLake_2111.tif|                0|    0|
| SeaLake_734.tif|                0|    0|
| SeaLake_430.tif|                0|    0|
|SeaLake_1243.tif|                0|    0|
| SeaLake_709.tif|                0|    0|
|SeaLake_1891.tif|                0|    0|
|   River_495.tif|                0|    0|
|SeaLake_2832.tif|                0|    0|
|SeaLake_2583.tif|                0|    0|
|SeaLake_2643.tif|                0|    0|
|  River_2095.tif|                0|    0|
|SeaLake_2582.tif|                0|    0|
|SeaLake_2657.tif|                0|    0|
|SeaLake_2834.tif|                0|    0|
|SeaLake_2317.tif|                0|    0|
|  River_2032.tif|                0|    0|
|SeaLake_1930.tif|                0|    0|
+----------

+----------------+-------------+-----+
|          origin|isTouch_river|class|
+----------------+-------------+-----+
|SeaLake_2847.tif|            0|    0|
| SeaLake_363.tif|            0|    0|
|  River_2019.tif|            0|    0|
|SeaLake_2111.tif|            0|    0|
| SeaLake_734.tif|            0|    0|
| SeaLake_430.tif|            0|    0|
|SeaLake_1243.tif|            0|    0|
| SeaLake_709.tif|            0|    0|
|SeaLake_1891.tif|            0|    0|
|   River_495.tif|            0|    0|
|SeaLake_2832.tif|            0|    0|
|SeaLake_2583.tif|            0|    0|
|SeaLake_2643.tif|            0|    0|
|  River_2095.tif|            0|    0|
|SeaLake_2582.tif|            0|    0|
|SeaLake_2657.tif|            0|    0|
|SeaLake_2834.tif|            0|    0|
|SeaLake_2317.tif|            0|    0|
|  River_2032.tif|            0|    0|
|SeaLake_1930.tif|            0|    0|
+----------------+-------------+-----+
only showing top 20 rows



In [35]:
df_16 = river_overlap.groupby("origin").sum("isOverlaps_river").alias("total_overlap_river")
df_17 = river_distance.groupby("origin").min("distance_river").alias("distance_river")
df_18 = river_contains.groupby("origin").sum("isContain_river").alias("total_contain_river")
df_19 = river_touches.groupby("origin").sum("isTouch_river").alias("total_touch_river")
df_20 = river_intersects.groupby("origin").sum("isIntersect_river").alias("total_intersect_river")

df16 = df_16.toPandas()
df17 = df_17.toPandas()
df18 = df_18.toPandas()
df19 = df_19.toPandas()
df20 = df_20.toPandas()

In [45]:
df_final = df16.merge(df17, on='origin', how="inner")
df_final = df18.merge(df_final, on="origin", how="inner")
df_final = df19.merge(df_final, on="origin", how="inner")
df_final = df20.merge(df_final, on="origin", how="inner")

df_final.to_csv("../data2/euro_sat/river.csv", index=False)

In [16]:
lakes_points = points.filter((points.attr_key=="water") & (points.attr_value=="lake"))
lakes_polygons = polygons.filter((polygons.attr_key=="water") & (polygons.attr_value=="lake"))
lakes_kb = lakes_points.union(lakes_polygons)
lakes_kb.persist().show()

lakes_kb.createOrReplaceTempView("lakes_kb")
lakes_overlap = spark.sql("select origin, cast(ST_Overlaps(train.geom, lakes_kb.Geometry) as INT) as isOverlaps_lake, class from train, lakes_kb") 
lakes_distance = spark.sql("select origin, ST_Distance(train.geom, lakes_kb.Geometry) as distance_lakes, class from train, lakes_kb")
lakes_contains = spark.sql("select origin, cast(ST_Contains(train.geom, lakes_kb.Geometry) as INT) as isContain_lakes, class from train, lakes_kb") 
lakes_intersects = spark.sql("select origin, cast(ST_Intersects(train.geom, lakes_kb.Geometry) as INT) as isIntersect_lakes, class from train, lakes_kb") 
lakes_touches = spark.sql("select origin, cast(ST_Touches(train.geom, lakes_kb.Geometry) as INT) as isTouch_lakes, class from train, lakes_kb") 

lakes_distance.persist().show()
lakes_overlap.persist().show()
lakes_contains.persist().show()
lakes_intersects.persist().show()
lakes_touches.persist().show()



22/04/23 20:50:15 WARN CacheManager: Asked to cache already cached data.
22/04/23 20:50:15 WARN CacheManager: Asked to cache already cached data.


+----------+--------------------+--------+----------+
|        id|            Geometry|attr_key|attr_value|
+----------+--------------------+--------+----------+
|3128843863|POINT (6.86592660...|   water|      lake|
|5896261328|POINT (25.7575144...|   water|      lake|
|5911625881|POINT (8.4015561 ...|   water|      lake|
|9646086926|POINT (11.361081 ...|   water|      lake|
|9646086927|POINT (11.3611132...|   water|      lake|
|9646086928|POINT (11.3611132...|   water|      lake|
|9646086929|POINT (11.3612205...|   water|      lake|
|9646086930|POINT (11.3612527...|   water|      lake|
|9646086931|POINT (11.3612044...|   water|      lake|
|9646086939|POINT (11.3618696...|   water|      lake|
|9075644143|POINT (10.3166767...|   water|      lake|
|9075725605|POINT (10.2393222...|   water|      lake|
|2373058259|POINT (43.4811062...|   water|      lake|
|2373058272|POINT (43.4830914...|   water|      lake|
|2373058526|POINT (43.4917781...|   water|      lake|
|9599461837|POINT (-1.116455

22/04/23 20:50:16 WARN CacheManager: Asked to cache already cached data.
22/04/23 20:50:16 WARN CacheManager: Asked to cache already cached data.


+----------------+---------------+-----+
|          origin|isContain_lakes|class|
+----------------+---------------+-----+
|SeaLake_2847.tif|              0|    0|
|SeaLake_2847.tif|              0|    0|
|SeaLake_2847.tif|              0|    0|
|SeaLake_2847.tif|              0|    0|
|SeaLake_2847.tif|              0|    0|
|SeaLake_2847.tif|              0|    0|
|SeaLake_2847.tif|              0|    0|
|SeaLake_2847.tif|              0|    0|
|SeaLake_2847.tif|              0|    0|
|SeaLake_2847.tif|              0|    0|
|SeaLake_2847.tif|              0|    0|
|SeaLake_2847.tif|              0|    0|
|SeaLake_2847.tif|              0|    0|
|SeaLake_2847.tif|              0|    0|
|SeaLake_2847.tif|              0|    0|
|SeaLake_2847.tif|              0|    0|
|SeaLake_2847.tif|              0|    0|
|SeaLake_2847.tif|              0|    0|
|SeaLake_2847.tif|              0|    0|
|SeaLake_2847.tif|              0|    0|
+----------------+---------------+-----+
only showing top

+----------------+-----------------+-----+
|          origin|isIntersect_lakes|class|
+----------------+-----------------+-----+
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
|SeaLake_2847.tif|                0|    0|
+----------

+----------------+-------------+-----+
|          origin|isTouch_lakes|class|
+----------------+-------------+-----+
|SeaLake_2847.tif|            0|    0|
|SeaLake_2847.tif|            0|    0|
|SeaLake_2847.tif|            0|    0|
|SeaLake_2847.tif|            0|    0|
|SeaLake_2847.tif|            0|    0|
|SeaLake_2847.tif|            0|    0|
|SeaLake_2847.tif|            0|    0|
|SeaLake_2847.tif|            0|    0|
|SeaLake_2847.tif|            0|    0|
|SeaLake_2847.tif|            0|    0|
|SeaLake_2847.tif|            0|    0|
|SeaLake_2847.tif|            0|    0|
|SeaLake_2847.tif|            0|    0|
|SeaLake_2847.tif|            0|    0|
|SeaLake_2847.tif|            0|    0|
|SeaLake_2847.tif|            0|    0|
|SeaLake_2847.tif|            0|    0|
|SeaLake_2847.tif|            0|    0|
|SeaLake_2847.tif|            0|    0|
|SeaLake_2847.tif|            0|    0|
+----------------+-------------+-----+
only showing top 20 rows



In [36]:
df_21 = lakes_overlap.groupby("origin").sum("isOverlaps_lake").alias("total_overlap_lake")
df_22 = lakes_distance.groupby("origin").min("distance_lakes").alias("distance_lake")
df_23 = lakes_contains.groupby("origin").sum("isContain_lakes").alias("total_contain_lake")
df_24 = lakes_touches.groupby("origin").sum("isTouch_lakes").alias("total_touch_lake")
df_25 = lakes_intersects.groupby("origin").sum("isIntersect_lakes").alias("total_intersect_lake")

df21 = df_21.toPandas()
df22 = df_22.toPandas()
df23 = df_23.toPandas()
df24 = df_24.toPandas()
df25 = df_25.toPandas()

In [47]:
df_final = df21.merge(df22, on='origin', how="inner")
df_final = df23.merge(df_final, on="origin", how="inner")
df_final = df24.merge(df_final, on="origin", how="inner")
df_final = df25.merge(df_final, on="origin", how="inner")

df_final.to_csv("../data2/euro_sat/lakes.csv", index=False)

In [55]:
df_final[df_final["sum(isContain_lakes)"]>0].head(10)

,origin,sum(isIntersect_lakes),sum(isTouch_lakes),sum(isContain_lakes),sum(isOverlaps_lake),min(distance_lakes)
238,Highway_1881.tif,1,0,1,0,0.0
1685,Forest_1735.tif,1,0,1,0,0.0
2784,River_2003.tif,1,0,1,0,0.0
2795,SeaLake_1166.tif,1,0,1,0,0.0
3415,Forest_196.tif,2,0,1,1,0.0
4286,Forest_734.tif,1,0,1,0,0.0
4522,Highway_30.tif,1,0,1,0,0.0
4560,Highway_1079.tif,1,0,1,0,0.0
5261,HerbaceousVegetation_1703.tif,1,0,1,0,0.0
5304,Industrial_109.tif,1,0,1,0,0.0


In [17]:
grasslands_points = points.filter((points.attr_key=="natural") & (points.attr_value=="grassland"))
grasslands_polygons = polygons.filter((polygons.attr_key=="natural") & (polygons.attr_value=="grassland"))
grasslands_kb = grasslands_points.union(grasslands_polygons)
grasslands_kb.persist().show()

grasslands_kb.createOrReplaceTempView("grasslands_kb")
grassland_overlap = spark.sql("select origin, cast(ST_Overlaps(train.geom, grasslands_kb.Geometry) as INT) as isOverlaps_grassland, class from train, grasslands_kb") 
grassland_distance = spark.sql("select origin, ST_Distance(train.geom, grasslands_kb.Geometry) as distance_grasslands, class from train,grasslands_kb")
grassland_contains = spark.sql("select origin, cast(ST_Contains(train.geom, grasslands_kb.Geometry) as INT) as isContain_grasslands, class from train, grasslands_kb") 
grassland_intersects = spark.sql("select origin, cast(ST_Intersects(train.geom, grasslands_kb.Geometry) as INT) as isIntersect_grasslands, class from train, grasslands_kb") 
grassland_touches = spark.sql("select origin, cast(ST_Touches(train.geom, grasslands_kb.Geometry) as INT) as isTouch_grasslands, class from train, grasslands_kb") 

grassland_distance.persist().show()
grassland_overlap.persist().show()
grassland_contains.persist().show()
grassland_intersects.persist().show()
grassland_touches.persist().show()


+----------+--------------------+--------+----------+
|        id|            Geometry|attr_key|attr_value|
+----------+--------------------+--------+----------+
|3136907158|POINT (9.51959400...| natural| grassland|
|1242833995|POINT (-6.6255221...| natural| grassland|
|1242860002|POINT (-6.6472476...| natural| grassland|
|1243387829|POINT (-3.2368313...| natural| grassland|
|2322292063|POINT (36.183979 ...| natural| grassland|
|3648585854|POINT (36.1921651...| natural| grassland|
|6372458069|POINT (6.16256880...| natural| grassland|
|8851596602|POINT (24.0672168...| natural| grassland|
|8958429988|POINT (19.0432736...| natural| grassland|
|8806651512|POINT (2.35468940...| natural| grassland|
|8806651513|POINT (2.3527787 ...| natural| grassland|
|7564341484|POINT (40.9019494...| natural| grassland|
|8151864033|POINT (10.6729227...| natural| grassland|
|8413025726|POINT (9.48333430...| natural| grassland|
|4283012817|POINT (17.6103240...| natural| grassland|
|3686726898|POINT (-6.958435

+----------------+-------------------+-----+
|          origin|distance_grasslands|class|
+----------------+-------------------+-----+
|SeaLake_2847.tif| 13.498639333894433|    0|
|SeaLake_2847.tif|  7.342695117141778|    0|
|SeaLake_2847.tif|  7.254121159977067|    0|
|SeaLake_2847.tif| 3.2818968814325866|    0|
|SeaLake_2847.tif| 37.968189429321896|    0|
|SeaLake_2847.tif|  38.22954992557997|    0|
|SeaLake_2847.tif| 23.985222764021355|    0|
|SeaLake_2847.tif|   28.2853212039889|    0|
|SeaLake_2847.tif|  21.06711692045823|    0|
|SeaLake_2847.tif| 3.6206033939888793|    0|
|SeaLake_2847.tif|  3.619652212598262|    0|
|SeaLake_2847.tif|  45.16686915985807|    0|
|SeaLake_2847.tif|   12.0757558387011|    0|
|SeaLake_2847.tif| 19.363595707544317|    0|
|SeaLake_2847.tif| 27.229262851510104|    0|
|SeaLake_2847.tif| 7.0987113894989164|    0|
|SeaLake_2847.tif| 22.722567343754044|    0|
|SeaLake_2847.tif| 15.326348164829094|    0|
|SeaLake_2847.tif| 14.232441045407793|    0|
|SeaLake_2

+----------------+--------------------+-----+
|          origin|isOverlaps_grassland|class|
+----------------+--------------------+-----+
|SeaLake_2847.tif|                   0|    0|
| SeaLake_363.tif|                   0|    0|
|  River_2019.tif|                   0|    0|
|SeaLake_2111.tif|                   0|    0|
| SeaLake_734.tif|                   0|    0|
| SeaLake_430.tif|                   0|    0|
|SeaLake_1243.tif|                   0|    0|
| SeaLake_709.tif|                   0|    0|
|SeaLake_1891.tif|                   0|    0|
|   River_495.tif|                   0|    0|
|SeaLake_2832.tif|                   0|    0|
|SeaLake_2583.tif|                   0|    0|
|SeaLake_2643.tif|                   0|    0|
|  River_2095.tif|                   0|    0|
|SeaLake_2582.tif|                   0|    0|
|SeaLake_2657.tif|                   0|    0|
|SeaLake_2834.tif|                   0|    0|
|SeaLake_2317.tif|                   0|    0|
|  River_2032.tif|                

+----------------+--------------------+-----+
|          origin|isContain_grasslands|class|
+----------------+--------------------+-----+
|SeaLake_2847.tif|                   0|    0|
| SeaLake_363.tif|                   0|    0|
|  River_2019.tif|                   0|    0|
|SeaLake_2111.tif|                   0|    0|
| SeaLake_734.tif|                   0|    0|
| SeaLake_430.tif|                   0|    0|
|SeaLake_1243.tif|                   0|    0|
| SeaLake_709.tif|                   0|    0|
|SeaLake_1891.tif|                   0|    0|
|   River_495.tif|                   0|    0|
|SeaLake_2832.tif|                   0|    0|
|SeaLake_2583.tif|                   0|    0|
|SeaLake_2643.tif|                   0|    0|
|  River_2095.tif|                   0|    0|
|SeaLake_2582.tif|                   0|    0|
|SeaLake_2657.tif|                   0|    0|
|SeaLake_2834.tif|                   0|    0|
|SeaLake_2317.tif|                   0|    0|
|  River_2032.tif|                

+----------------+----------------------+-----+
|          origin|isIntersect_grasslands|class|
+----------------+----------------------+-----+
|SeaLake_2847.tif|                     0|    0|
| SeaLake_363.tif|                     0|    0|
|  River_2019.tif|                     0|    0|
|SeaLake_2111.tif|                     0|    0|
| SeaLake_734.tif|                     0|    0|
| SeaLake_430.tif|                     0|    0|
|SeaLake_1243.tif|                     0|    0|
| SeaLake_709.tif|                     0|    0|
|SeaLake_1891.tif|                     0|    0|
|   River_495.tif|                     0|    0|
|SeaLake_2832.tif|                     0|    0|
|SeaLake_2583.tif|                     0|    0|
|SeaLake_2643.tif|                     0|    0|
|  River_2095.tif|                     0|    0|
|SeaLake_2582.tif|                     0|    0|
|SeaLake_2657.tif|                     0|    0|
|SeaLake_2834.tif|                     0|    0|
|SeaLake_2317.tif|                     0

In [56]:
df_26 = grassland_overlap.groupby("origin").sum("isOverlaps_grassland").alias("total_overlap_lake")
df_27 = grassland_distance.groupby("origin").min("distance_grasslands").alias("distance_lake")
df_28 = grassland_contains.groupby("origin").sum("isContain_grasslands").alias("total_contain_lake")
df_29 = grassland_touches.groupby("origin").sum("isTouch_grasslands").alias("total_touch_lake")
df_30 = grassland_intersects.groupby("origin").sum("isIntersect_grasslands").alias("total_intersect_lake")

df26 = df_26.toPandas()
df27 = df_27.toPandas()
df28 = df_28.toPandas()
df29 = df_29.toPandas()
df30 = df_30.toPandas()

df_final = df26.merge(df27, on='origin', how="inner")
df_final = df28.merge(df_final, on="origin", how="inner")
df_final = df29.merge(df_final, on="origin", how="inner")
df_final = df30.merge(df_final, on="origin", how="inner")

df_final.to_csv("../data2/euro_sat/grasslands.csv", index=False)

In [28]:
highway_points = points.filter(points.attr_key=="highway")
highway_polygons = polygons.filter(polygons.attr_key=="highway")
highway_kb = highway_points.union(highway_polygons)
highway_kb.persist().show()
geo_train_df.createOrReplaceTempView("train")


highway_kb.createOrReplaceTempView("highway_kb")
highway_overlap = spark.sql("select origin, cast(ST_Overlaps(train.geom, highway_kb.Geometry) as INT) as isOverlaps_highway, class from train, highway_kb") 
highway_distance = spark.sql("select origin, ST_Distance(train.geom, highway_kb.Geometry) as distance_highway, class from train,highway_kb")
highway_contains = spark.sql("select origin, cast(ST_Contains(train.geom, highway_kb.Geometry) as INT) as isContain_highways, class from train, highway_kb") 
highway_intersects = spark.sql("select origin, cast(ST_Intersects(train.geom, highway_kb.Geometry) as INT) as isIntersect_highways, class from train, highway_kb") 
highway_touches = spark.sql("select origin, cast(ST_Touches(train.geom, highway_kb.Geometry) as INT) as isTouch_highways, class from train, highway_kb") 

highway_distance.persist().show()
highway_overlap.persist().show()
highway_contains.persist().show()
highway_intersects.persist().show()
highway_touches.persist().show()

df_31 = highway_overlap.groupby("origin").sum("isOverlaps_highway").alias("total_overlap_lake")
df_32 = highway_distance.groupby("origin").min("distance_highway").alias("distance_lake")
df_33 = highway_contains.groupby("origin").sum("isContain_highways").alias("total_contain_lake")
df_34 = highway_touches.groupby("origin").sum("isTouch_highways").alias("total_touch_lake")
df_35 = highway_intersects.groupby("origin").sum("isIntersect_highways").alias("total_intersect_lake")

df31 = df_31.toPandas()
df32 = df_32.toPandas()
df33 = df_33.toPandas()
df34 = df_34.toPandas()
df35 = df_35.toPandas()

df_final = df31.merge(df32, on='origin', how="inner")
df_final = df33.merge(df_final, on="origin", how="inner")
df_final = df34.merge(df_final, on="origin", how="inner")
df_final = df35.merge(df_final, on="origin", how="inner")

df_final.to_csv("../data2/euro_sat/highways.csv", index=False)

22/04/27 17:55:20 WARN CacheManager: Asked to cache already cached data.


+---------+--------------------+--------+---------------+
|       id|            Geometry|attr_key|     attr_value|
+---------+--------------------+--------+---------------+
|340497091|POINT (24.9726741...| highway|       crossing|
|340497126|POINT (24.9696257...| highway|       bus_stop|
|340497179|POINT (-3.7021749...| highway|       crossing|
|340498257|POINT (10.2724356...| highway| turning_circle|
|340498741|POINT (-1.7144941...| highway|           stop|
|340499204|POINT (10.6091171...| highway| turning_circle|
|340499528|POINT (6.41288860...| highway|       crossing|
|340500976|POINT (-0.4695351...| highway|traffic_signals|
|340505216|POINT (6.0935505 ...| highway|       crossing|
|340505304|POINT (6.09609780...| highway| turning_circle|
|340505336|POINT (6.0932594 ...| highway| turning_circle|
|340505410|POINT (6.0921151 ...| highway| turning_circle|
|340505437|POINT (6.09917080...| highway|       crossing|
|340505468|POINT (6.1072534 ...| highway| turning_circle|
|340505489|POI

Exception in thread "refresh progress"                                          java.lang.OutOfMemoryError: GC overhead limit exceeded
22/04/27 18:36:27 WARN TransportChannelHandler: Exception in connection from /10.218.108.131:32816
java.lang.OutOfMemoryError: GC overhead limit exceeded
	at java.util.regex.Pattern.compile(Pattern.java:1693)
	at java.util.regex.Pattern.<init>(Pattern.java:1352)
	at java.util.regex.Pattern.compile(Pattern.java:1028)
	at org.apache.spark.network.util.JavaUtils.timeStringAs(JavaUtils.java:232)
	at org.apache.spark.network.util.JavaUtils.timeStringAsSec(JavaUtils.java:269)
	at org.apache.spark.util.Utils$.timeStringAsSeconds(Utils.scala:1179)
	at org.apache.spark.rpc.RpcTimeout$.apply(RpcTimeout.scala:131)
	at org.apache.spark.util.RpcUtils$.askRpcTimeout(RpcUtils.scala:51)
	at org.apache.spark.rpc.RpcEndpointRef.<init>(RpcEndpointRef.scala:35)
	at org.apache.spark.rpc.netty.NettyRpcEndpointRef.<init>(NettyRpcEnv.scala:534)
	at org.apache.spark.rpc.netty.R

Py4JJavaError: An error occurred while calling o2295.showString.
: java.util.concurrent.ExecutionException: org.apache.spark.util.SparkFatalException
	at java.util.concurrent.FutureTask.report(FutureTask.java:122)
	at java.util.concurrent.FutureTask.get(FutureTask.java:206)
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:194)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeBroadcast$1(SparkPlan.scala:193)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apache.spark.sql.execution.SparkPlan.executeBroadcast(SparkPlan.scala:189)
	at org.apache.spark.sql.execution.joins.BroadcastNestedLoopJoinExec.doExecute(BroadcastNestedLoopJoinExec.scala:350)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execut

util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
22/04/27 18:36:27 WARN StandaloneAppClient$ClientEndpoint: Connection to EN4119508L.cidse.dhcp.asu.edu:7077 failed; waiting for master to reconnect...
